# Data harmoinsation in imaging - the problem
This notebook will give a brief example of how differences can arise when acquiring data using multiple scanners in a study. For more background on this, please see the documenation web pages. 

To illistrate this probelm, let's take a scenario where we expect the least amount of change. We will jave a hjealthy subejct scanned on two different scanners. We will register them together and see if there are any differences present. if the scans produced were truly quantitiave, then we would expect to see a minimal difference between the scans save for a bit of measurement error. 

## Setup
To start with, we will import a few basic packages and setup the notebook to have interactive content (the matplotlib widget *magic* command in Jupyter)

In [1]:
import os
import shutil
import matplotlib
%matplotlib widget

## Getting the data
For this demonstration, I used [Datalad](https://www.datalad.org/) to setup the data repository. Datalad is built upon git and git-annex, and it provides version control for data management, data sharing and reproducible science. For this lesson, it offers us a convenient way of grabbing the data that we need. These comamands import datalad so that we can use it to clone the repository into our virtual machine.

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import datalad.api as dl

Next we "clone" the repository to a local destination. No big files are downloaded yet, just the metadata so this computer know what data exists.

In [4]:
dl_source='https://github.com/HealthBioscienceIDEAS/demon-imaging-data.git'
sample=dl.clone(dl_source,path='/tmp/sample',description='Cloned sample dataset for import')
sample.update(merge=True)
sample.siblings(action='enable',name='demons-data')

[INFO] Fetching updates for Dataset(/tmp/sample) 


.: demons-data(?) [git]


[{'action': 'enable-sibling',
  'path': '/tmp/sample',
  'type': 'sibling',
  'name': 'demons-data',
  'refds': '/tmp/sample',
  'status': 'ok'}]

Now we get our first and second image from the same individual using the datalad get command. Big data is not downloaded automaticaly, only as it is needed, saving valuable space, especially on large repositories. If not downloaded before, then this might take a few seconds on the binder instance, so be patient. When it is running, the number to the left becomes a star. When it is complete, it becomes a number again.

In [5]:
bl_img=sample.get('./baseline_t1.nii.gz')

In [6]:
fu_img=sample.get('./followup_t1.nii.gz')

# Intrasubject registration
The python package [nipype](https://nipype.readthedocs.io/en/latest/index.html) provides an effective means in Python to run your image processing workflows, taking "building blocks" of individual commands, allowing you to piece together bits of various software packages (FSL, SPM, FreeSurfer, AFNI, MRtrix, etc) into complete pipelines. We are just going to use it for simple cases here by registereing our baseline and followup image.

In [7]:
from nipype.interfaces import niftyreg


This sets up the node that will run rigid registration using  [NiftyReg](https://github.com/KCL-BMEIS/niftyreg), a lightweight easy to use registration package. We set the inputs and outputs up, and it will generate the command line, which you can see below.

In [8]:
out_path=os.path.join(os.getcwd(),'results')
if not os.path.exists(out_path):
    os.makedirs(out_path)
followup_aff=os.path.join(out_path,'followup_to_baseline_aff.txt')
followup_res=os.path.join(out_path,'followup_t1_resampled.nii.gz')
node = niftyreg.RegAladin(verbosity_off_flag=True)
node.inputs.ref_file = bl_img[0]['path']
node.inputs.flo_file = fu_img[0]['path']
node.inputs.aff_file = followup_aff
node.inputs.res_file = followup_res
node.cmdline

'reg_aladin -aff /Users/davecash/src/demon-imaging-harmonisation/results/followup_to_baseline_aff.txt -flo /tmp/sample/followup_t1.nii.gz -omp 1 -ref /tmp/sample/baseline_t1.nii.gz -res /Users/davecash/src/demon-imaging-harmonisation/results/followup_t1_resampled.nii.gz -voff'

If we are happy with that, then we run the code. This might take a little while, as nipype doesn't produce the output until it is all complete. 

In [9]:
node.run()

210913-10:33:19,311 nipype.interface INFO:
	 stdout 2021-09-13T10:33:19.310011:[NiftyReg WARNING] NiftyReg has not been compiled with OpenMP, the '-omp' flag is ignored


It has now generated the affine transformation between the two images and ar sampled followup image. You can now see these in the results directory of your drive and the next step will put the baseline there to, so that you can compare them on your own macine if you prefer. 

In [10]:
baseline_nii=os.path.join(out_path,'baseline_t1.nii.gz')
shutil.copyfile(bl_img[0]['path'],baseline_nii)

'/Users/davecash/src/demon-imaging-harmonisation/results/baseline_t1.nii.gz'

## Checking the registration

After the registration. images should be aligned such that the corresponding anatomy in the scans are aligned. If you look at the tree view of files, you should see the results directory and the files in there. The best way to check the registration is to open them up in your favourite iamge viewer like fsleyes that allows you to go back and forth between the baseline scan and the followup scan, so you can see the diferences. You can also look at them through this noteboook below using a lightweight 3D slice viewer called [nanslice](https://github.com/spinicist/nanslice).

In [11]:
import nanslice.jupyter as ns

Let's first look at the baseline in the viewer. Scroll around on each of the slices. You can change the brightness and contrast by modifying the clim argument below.   

In [25]:
base=ns.Layer(baseline_nii,clim=(30,350),label="Baseline")
ns.three_plane(base,cbar=True,interactive=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now for the repeat - The original clim arguments should produce similar greyscales for both. Make sure the slice sliders have the same location for both viewers.  
**What differences do you notice?**

In [23]:
repeat=ns.Layer(followup_res,clim=(20,220),label="Followup")
ns.three_plane(repeat,cbar=True,interactive=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In some cases, changing scanners can result in subtle geometric distortions between scans. In some cases, these changs in shape can appear as changes in volume that are not based on the pathophysiology. This animated GIF Here is an animation of the above scans so that you can see it as an example. It also further shows the differences in contrast and intensity between scans. These three effects are some of the main reasons for why data harmonisation between sites are needed. 
![DistortionExample](registration.gif "registration")